# 03. Model Stability
Compare OLS vs SVD solutions.

In [ ]:
import os
if not os.path.exists('Credit-Risk-Algebraic-ML'):
    !git clone https://github.com/adriangonz-afk/Credit-Risk-Algebraic-ML.git
os.chdir('Credit-Risk-Algebraic-ML')
print(f'✅ Entorno listo en: {os.getcwd()}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
X = np.load('data/processed/X_matrix.npy')
y = np.load('data/processed/y_vector.npy')
X_bias = np.c_[np.ones((X.shape[0], 1)), X]
try:
    theta_ols = np.linalg.inv(X_bias.T @ X_bias) @ X_bias.T @ y
except: theta_ols = np.zeros(X_bias.shape[1])
theta_svd = np.linalg.pinv(X_bias) @ y
plt.plot(theta_ols, alpha=0.5, label='OLS'); plt.plot(theta_svd, 'r', label='SVD'); plt.legend(); plt.show()